In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
import re

In [2]:
store_df = pd.read_csv('db/apps.csv',error_bad_lines=False)
store_df = store_df[['name','numDownloadsMin','aggregateRating','url']]
store_df.count()
#store_df.loc[apps_df['name'].str.contains('Twitter',na=False,flags=re.I,regex=True)]

b'Skipping line 304238: expected 11 fields, saw 12\n'
/home/ahmed/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


name               399381
numDownloadsMin    403905
aggregateRating    403905
url                403898
dtype: int64

In [5]:
apps_df  = pd.read_csv('db/Google-Playstore-Full.csv')
apps_df  = apps_df[['App Name','Rating','Installs']]
apps_df.count()
#apps_df.loc[apps_df['App Name'].str.contains('global citizen',na=False,flags=re.I,regex=True)]

/home/ahmed/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,3,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


App Name    267051
Rating      267052
Installs    267052
dtype: int64

In [6]:
#Joining the two data bases
all_df = apps_df.merge(store_df,how='left',left_on='App Name',right_on='name')
all_df = all_df[['App Name','Rating','Installs','url']]
#new.loc[new['App Name'].str.contains('global citizen',na=False,flags=re.I,regex=True)]

In [7]:
all_df['Developers Website'] = np.nan
all_df.head()

,App Name,Rating,Installs,url,Developers Website
0,DoorDash - Food Delivery,4.548561573,"5,000,000+",NaN,NaN
1,TripAdvisor Hotels Flights Restaurants Attract...,4.400671482,"100,000,000+",NaN,NaN
2,Peapod,3.656329393,"100,000+",NaN,NaN
3,foodpanda - Local Food Delivery,4.107232571,"10,000,000+",NaN,NaN
4,My CookBook Pro (Ad Free),4.647752285,"10,000+",https://play.google.com/store/apps/details?id=...,NaN


In [8]:
input_series  = pd.read_excel('Sourcing Input List.xlsx')
company_names = input_series['Company Name'].values
company_names

array(['Global Citizen', 'Twitter', 'SiriusXM', 'iHeartRadio',
       'JOANN Stores', 'LogMeIn', "Jersey Mike's Subs", 'iNTEL SECURITY',
       'Canva', 'Amanotes', 'Kdan Mobile', 'Zappos Family of Companies',
       'edX', 'Inkitt', 'APPLEBEE S', 'Coinbase', 'LevelUp', 'KABB-TV',
       'Popeyes Louisiana Kitchen, Inc.', 'Panda Restaurant Group',
       'First Orion', 'LightInTheBox', 'Bandsintown Group', 'Trigone',
       'Kahoot!', 'City Harvest', 'Severex',
       'Missouri Department of Conservation', 'Amanotes', 'Easybrain',
       'Aaptiv', 'Wave Systems Corp.', 'BuzzFeed', 'Miniclip',
       'Geeko Tech', 'Edmodo', 'MasterClass'], dtype=object)

In [27]:
int(''.join(all_df.at[0,'Installs'].split('+')[0].split(',')))

5000000

In [95]:
def create_output(company_names):
    i=0
    for num,name in enumerate(company_names):
        if(i == len(company_names)):
            break
        found = all_df.loc[all_df['App Name'].str.contains(name,na=False)][:10]
        #If Several : take the one with highest downloading rate - I could have added several rows 
        #But found it more convenient this way
        found['Installs'] = found['Installs'].map(lambda Installs: float(''.join(Installs.split('+')[0].split(','))))
        
        if(found['App Name'].count() >= 1):
            yield found.loc[found['Installs'].idxmax()],num
        else :
            yield found,num
        i += 1

out_series = pd.DataFrame({'Company':company_names,'App Name': np.nan,'Rating':np.nan,'Installs':np.nan,'url':np.nan,'Developers Website':np.nan})


for i,num in create_output(company_names):
    if(i.empty):
        continue
    out_series.iloc[num,1:] = i.values 

out_series.reset_index(drop=True)


,Company,App Name,Rating,Installs,url,Developers Website
0,Global Citizen,Global Citizen,3.726547003,100000.0,NaN,NaN
1,Twitter,Twitter,4.311233997,500000000.0,https://play.google.com/store/apps/details?id=...,NaN
2,SiriusXM,SiriusXM,4.316278458,10000000.0,NaN,NaN
3,iHeartRadio,"iHeartRadio - Free Music, Radio & Podcasts",4.661151886,50000000.0,NaN,NaN
4,JOANN Stores,NaN,NaN,NaN,NaN,NaN
5,LogMeIn,LogMeIn,3.966305494,1000000.0,https://play.google.com/store/apps/details?id=...,NaN
6,Jersey Mike's Subs,NaN,NaN,NaN,NaN,NaN
7,iNTEL SECURITY,NaN,NaN,NaN,NaN,NaN
8,Canva,"Canva: Graphic Design & Logo, Flyer, Poster maker",4.759311199,10000000.0,NaN,NaN
9,Amanotes,NaN,NaN,NaN,NaN,NaN


In [96]:
all_df.loc[all_df['App Name'].str.contains('Coinbase',na=False)]

,App Name,Rating,Installs,url,Developers Website
67664,Coinbase � Buy & Sell Bitcoin. Crypto Wallet.,4.256000519,"5,000,000+",NaN,NaN
3887559,AnyX Coinbase Pro App,4.214285851,"1,000+",NaN,NaN
4651836,ProfitTrading For Coinbase and Coinbase PRO,4.388888836,"1,000+",NaN,NaN
